https://huggingface.co/docs/transformers/en/training

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

In [ ]:
def tokenized_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenized_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

# Train in native pytorch

In [ ]:
import torch
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

In [ ]:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [ ]:
tokenized_datasets.set_format("torch")

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased")

In [ ]:
from torch.optim import AdamW

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

In [ ]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_cheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch

device = torch.device("cpu")
model.to(device)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
progess_bar = tqdm(range(num_training_steps))

In [ ]:
model.train()

In [ ]:
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v, in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()